In [170]:
import pandas as pd
import numpy as np
import ml_metrics 

In [171]:
df_train = pd.read_csv('train.csv')
df_test = pd.read_csv('test.csv')

In [172]:
sub = pd.read_csv('sample_submission.csv')

In [173]:
df_train.shape,df_test.shape

((37748, 9), (20327, 8))

In [174]:
import ast

In [175]:
df_train.Product_Holding_B1=df_train.Product_Holding_B1.apply(ast.literal_eval)
df_train.Product_Holding_B2=df_train.Product_Holding_B2.apply(ast.literal_eval)

df_test.Product_Holding_B1=df_test.Product_Holding_B1.apply(ast.literal_eval)

In [176]:
df_test['Product_Holding_B2']=np.nan

In [177]:
df=df_train.append(df_test,ignore_index=True)

In [178]:
df.shape

(58075, 9)

In [179]:
df2=pd.DataFrame(df.Product_Holding_B1.to_list(),index=df.Customer_ID).stack().reset_index().pivot_table(index=['Customer_ID'], columns=0, fill_value=0, aggfunc='size').reset_index()
df2.head()


,Customer_ID,P00,P1,P10,P11,P12,P13,P14,P15,P16,...,P2,P20,P21,P3,P4,P5,P6,P7,P8,P9
0,CC198719,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,CC198721,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,1,0
2,CC198726,0,0,1,0,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,CC198730,0,0,0,0,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,CC198733,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,1,0


In [180]:
X = df2.drop('Customer_ID',1).values
X.shape

(58075, 22)

In [181]:
import sklearn
from sklearn.decomposition import TruncatedSVD
num=5
SVD = TruncatedSVD(n_components=num, random_state=17)
matrix = SVD.fit_transform(X)
matrix.shape

(58075, 5)

In [182]:
X=pd.DataFrame(matrix,columns=['CustFeat'+str(i) for i in np.arange(1,num+1)])
X['Customer_ID']=df2['Customer_ID']
df2=df2.merge(X,on='Customer_ID')
df2.shape

(58075, 28)

In [183]:
X = df2.drop('Customer_ID',1).values.T
num=10
SVD = TruncatedSVD(n_components=num, random_state=17)
matrix = SVD.fit_transform(X)
matrix.shape

(27, 10)

In [184]:
X=pd.DataFrame(matrix,columns=['PFeat'+str(i) for i in np.arange(1,num+1)])
X['Program']=df2.drop('Customer_ID',1).columns
X.head()

,PFeat1,PFeat2,PFeat3,PFeat4,PFeat5,PFeat6,PFeat7,PFeat8,PFeat9,PFeat10,Program
0,-4.932651e-13,-9.271733e-12,-9.584207e-10,6.924331e-10,-8.112980e-11,9.102653e-15,-5.233083e-15,-2.710769e-14,1.672541e-15,-2.383802e-13,P00
1,-1.885413e-20,3.949864e-19,1.129908e-17,-1.410084e-18,-6.447180e-17,-9.365070e-15,7.382076e-14,-2.968934e-13,1.848936e-14,7.846398e-13,P1
2,1.077852e+01,5.588146e+00,-4.223465e+00,3.974076e+00,8.691444e+00,-5.519251e-02,3.057025e+00,-1.318456e+00,2.123631e+00,4.442167e+01,P10
3,1.054342e+00,9.243749e-01,-6.702210e-02,3.694152e+00,4.775701e+00,5.340598e-01,7.423637e+00,7.874302e+00,5.268711e+01,-2.278935e+00,P11
4,5.173653e+01,2.642360e+01,-6.478903e+00,2.989079e+01,7.116979e+01,3.482911e+00,1.424623e+00,-7.627436e-01,-4.077320e+00,-6.005129e+00,P12


In [185]:
y=pd.melt(df2, id_vars=['Customer_ID'])
y=y[y.value==1].drop('value',1).reset_index(drop=True)
y.columns=['Customer_ID','Program']
y.head()

,Customer_ID,Program
0,CC198751,P00
1,CC198873,P00
2,CC199405,P00
3,CC199423,P00
4,CC199426,P00


In [186]:
y=y.merge(X,on='Program')

In [187]:
y.head()

,Customer_ID,Program,PFeat1,PFeat2,PFeat3,PFeat4,PFeat5,PFeat6,PFeat7,PFeat8,PFeat9,PFeat10
0,CC198751,P00,-4.932651e-13,-9.271733e-12,-9.584207e-10,6.924331e-10,-8.112980e-11,9.102653e-15,-5.233083e-15,-2.710769e-14,1.672541e-15,-2.383802e-13
1,CC198873,P00,-4.932651e-13,-9.271733e-12,-9.584207e-10,6.924331e-10,-8.112980e-11,9.102653e-15,-5.233083e-15,-2.710769e-14,1.672541e-15,-2.383802e-13
2,CC199405,P00,-4.932651e-13,-9.271733e-12,-9.584207e-10,6.924331e-10,-8.112980e-11,9.102653e-15,-5.233083e-15,-2.710769e-14,1.672541e-15,-2.383802e-13
3,CC199423,P00,-4.932651e-13,-9.271733e-12,-9.584207e-10,6.924331e-10,-8.112980e-11,9.102653e-15,-5.233083e-15,-2.710769e-14,1.672541e-15,-2.383802e-13
4,CC199426,P00,-4.932651e-13,-9.271733e-12,-9.584207e-10,6.924331e-10,-8.112980e-11,9.102653e-15,-5.233083e-15,-2.710769e-14,1.672541e-15,-2.383802e-13


In [188]:
y=y.drop('Program',1).groupby('Customer_ID').mean().reset_index()
y.head()

,Customer_ID,PFeat1,PFeat2,PFeat3,PFeat4,PFeat5,PFeat6,PFeat7,PFeat8,PFeat9,PFeat10
0,CC198719,147.585418,-57.641096,-16.104555,-23.449705,-8.499169,-2.948053,-7.219103,-4.493826,2.134244,-0.796968
1,CC198721,69.496216,-11.080375,22.467182,-13.282896,-4.117063,-6.073776,17.721350,2.580272,-1.885691,-0.521345
2,CC198726,70.033488,-8.543116,-8.935641,3.471719,23.787356,0.159889,-0.912485,-2.191675,0.060185,12.539857
3,CC198730,99.660974,-15.608748,-11.291729,3.220540,31.335311,0.267429,-2.897240,-2.628285,-0.971538,-3.401048
4,CC198733,69.496216,-11.080375,22.467182,-13.282896,-4.117063,-6.073776,17.721350,2.580272,-1.885691,-0.521345


In [189]:
df2=df2.merge(y,on='Customer_ID')

In [190]:
df=df.merge(df2,on='Customer_ID')
df.head()

,Customer_ID,Gender,Age,Vintage,Is_Active,City_Category,Customer_Category,Product_Holding_B1,Product_Holding_B2,P00,...,PFeat1,PFeat2,PFeat3,PFeat4,PFeat5,PFeat6,PFeat7,PFeat8,PFeat9,PFeat10
0,CC264719,Male,41,14,0,C1,S3,[P16],[P8],0,...,37.437148,118.675812,-18.394389,-36.489265,-9.349696,-1.830386,-7.893067,-3.568102,2.170324,-1.000623
1,CC209679,Female,47,14,1,C1,S2,"[P13, P20]",[P3],0,...,85.466365,-23.842804,-3.710319,-5.578018,-11.120753,32.443915,-0.662519,-3.267107,0.763914,-0.407615
2,CC319633,Female,59,14,0,C2,S2,[P11],[P00],0,...,1.054342,0.924375,-0.067022,3.694152,4.775701,0.534060,7.423637,7.874302,52.687108,-2.278935
3,CC231413,Female,32,16,0,C1,S2,"[P8, P13]",[P6],0,...,82.052064,-26.793491,-12.425405,-20.115202,-4.614348,-3.869508,24.788109,4.646104,-2.803347,-1.229480
4,CC259633,Male,30,15,0,C2,S3,"[P16, P17, P21]","[P8, P12]",0,...,42.664887,56.393996,19.228720,11.218587,-16.399951,-9.605442,0.125934,-1.340766,0.328274,-0.372141


In [191]:
cols=['Gender','City_Category','Customer_Category']

from sklearn import preprocessing
for col in cols:
    le = preprocessing.LabelEncoder()
    df[col]=le.fit_transform(df[col])

In [192]:
target=pd.DataFrame(df_train.Product_Holding_B2.to_list(),index=df_train.Customer_ID).stack().reset_index().drop('level_1',1)
target.columns=['Customer_ID','target']
print(target.shape)
target.head()


(52057, 2)


,Customer_ID,target
0,CC264719,P8
1,CC209679,P3
2,CC319633,P00
3,CC231413,P6
4,CC259633,P8


In [193]:
df_train2=df.drop(['Product_Holding_B1','Product_Holding_B2'],1).merge(target,on='Customer_ID')
print(df_train2.shape)
df_train2.head()

(52057, 45)


,Customer_ID,Gender,Age,Vintage,Is_Active,City_Category,Customer_Category,P00,P1,P10,...,PFeat2,PFeat3,PFeat4,PFeat5,PFeat6,PFeat7,PFeat8,PFeat9,PFeat10,target
0,CC264719,1,41,14,0,0,2,0,0,0,...,118.675812,-18.394389,-36.489265,-9.349696,-1.830386,-7.893067,-3.568102,2.170324,-1.000623,P8
1,CC209679,0,47,14,1,0,1,0,0,0,...,-23.842804,-3.710319,-5.578018,-11.120753,32.443915,-0.662519,-3.267107,0.763914,-0.407615,P3
2,CC319633,0,59,14,0,1,1,0,0,0,...,0.924375,-0.067022,3.694152,4.775701,0.534060,7.423637,7.874302,52.687108,-2.278935,P00
3,CC231413,0,32,16,0,0,1,0,0,0,...,-26.793491,-12.425405,-20.115202,-4.614348,-3.869508,24.788109,4.646104,-2.803347,-1.229480,P6
4,CC259633,1,30,15,0,1,2,0,0,0,...,56.393996,19.228720,11.218587,-16.399951,-9.605442,0.125934,-1.340766,0.328274,-0.372141,P8


In [194]:
le_target = preprocessing.LabelEncoder()
df_train2['target']=le_target.fit_transform(df_train2['target'])

In [195]:
df_train2.target.nunique()

20

In [196]:
df_test2=df.drop(['Product_Holding_B1','Product_Holding_B2'],1).merge(df_test[['Customer_ID']],on='Customer_ID')
print(df_test2.shape)
df_test2.head()

(20327, 44)


,Customer_ID,Gender,Age,Vintage,Is_Active,City_Category,Customer_Category,P00,P1,P10,...,PFeat1,PFeat2,PFeat3,PFeat4,PFeat5,PFeat6,PFeat7,PFeat8,PFeat9,PFeat10
0,CC372708,0,31,31,0,1,2,0,0,0,...,99.660974,-15.608748,-11.291729,3.220540,31.335311,0.267429,-2.897240,-2.628285,-0.971538,-3.401048
1,CC216072,1,28,37,1,0,1,0,0,0,...,99.660974,-15.608748,-11.291729,3.220540,31.335311,0.267429,-2.897240,-2.628285,-0.971538,-3.401048
2,CC387629,1,31,12,0,1,2,0,0,0,...,23.347312,9.955488,8.683917,12.293669,-13.742337,67.835883,5.894065,-2.040387,-0.606415,-0.018262
3,CC389228,0,55,11,0,1,1,0,0,0,...,96.879205,-13.740389,-16.138182,23.156802,-22.613418,-9.725839,-1.268033,-1.698315,0.499561,-0.903846
4,CC394445,1,51,49,1,1,0,0,0,0,...,147.585418,-57.641096,-16.104555,-23.449705,-8.499169,-2.948053,-7.219103,-4.493826,2.134244,-0.796968


In [197]:
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error,roc_auc_score,precision_score

In [198]:
#train-test-split
train_id,test_id=train_test_split(df_train[['Customer_ID']],test_size=0.3,random_state=0,shuffle=True)

In [199]:
feat=list(df_train2.columns.drop(['Customer_ID','target']))

In [200]:
#Converting the dataset in proper LGB format
tr=df_train2[df_train2.Customer_ID.isin(list(train_id.Customer_ID))]
d_train=lgb.Dataset(tr[feat], label=tr.target)


In [201]:
df_train2.target.nunique()

20

In [202]:
#setting up the parameters
params={'learning_rate':0.01,
       'boosting_type':'gbdt',
       'objective':'multiclass',
       'metric':'multi_logloss',
       'max_depth':6,
       'num_class':20}

#training the model
clf=lgb.train(params,d_train,500)  #training the model on 100 epocs
#prediction on the test dataset
y_pred_1=clf.predict(df_test2[feat])

[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.006681 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 3697
[LightGBM] [Info] Number of data points in the train set: 36418, number of used features: 42
[LightGBM] [Info] Start training from score -2.170751
[LightGBM] [Info] Start training from score -2.754358
[LightGBM] [Info] Start training from score -2.892956
[LightGBM] [Info] Start training from score -5.658631
[LightGBM] [Info] Start training from score -1.972314
[LightGBM] [Info] Start training from score -3.533968
[LightGBM] [Info] Start training from score -8.104923
[LightGBM] [Info] Start training from score -7.283943
[LightGBM] [Info

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

In [203]:
te=df.merge(test_id,on='Customer_ID')

In [204]:
y_pred_2 = ([list(le_target.inverse_transform(line.argsort()[-3:][::-1])) for line in clf.predict(te[feat])])
te['Product_Holding_B2_predict']=y_pred_2
print(ml_metrics.mapk(te.Product_Holding_B2, te.Product_Holding_B2_predict, 3))
#0.6944518027961736

0.694625950453765


In [ ]:
#training the model
#Converting the dataset in proper LGB format
d_train=lgb.Dataset(df_train2[feat], label=df_train2.target)

clf=lgb.train(params,d_train,600)  #training the model on 100 epocs
#prediction on the test dataset
y_pred_1=clf.predict(df_test2[feat])

[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.007564 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 3718
[LightGBM] [Info] Number of data points in the train set: 52057, number of used features: 42
[LightGBM] [Info] Start training from score -2.176032
[LightGBM] [Info] Start training from score -2.755391
[LightGBM] [Info] Start training from score -2.882813
[LightGBM] [Info] Start training from score -5.634348
[LightGBM] [Info] Start training from score -1.966522
[LightGBM] [Info] Start training from score -3.550882
[LightGBM] [Info] Start training from score -7.864362
[LightGBM] [Info] Start training from score -7.146522
[LightGBM] [Info

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf


In [ ]:
y_pred_2 = ([list(le_target.inverse_transform(line.argsort()[-3:][::-1])) for line in y_pred_1])
df_test2['Product_Holding_B2']=y_pred_2



In [ ]:
df_test2[['Customer_ID','Product_Holding_B2']].to_csv('fr5.csv',index=False)